In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from ucsd_dataset import UCSDAnomalyDataset
from video_CAE import VideoAutoencoderLSTM
import torch.backends.cudnn as cudnn

In [2]:
#from video_CAE import VideoAutoencoderLSTM
model = VideoAutoencoderLSTM()
criterion = nn.MSELoss()

use_cuda = torch.cuda.is_available()
if use_cuda:
    cudnn.benchmark = True
    model.set_cuda()
    criterion.cuda()
    
train_ds = UCSDAnomalyDataset('./data/UCSD_Anomaly_Dataset.v1p2/UCSDped1/Train', time_stride=3)
train_dl = data.DataLoader(train_ds, batch_size=32, shuffle=True)

optimizer = optim.Adam(model.parameters(), lr=1e-4, eps=1e-6, weight_decay=1e-5)

for epoch in range(100):
    for batch_idx, x in enumerate(train_dl):
        optimizer.zero_grad()
        if use_cuda:
            x = x.cuda()
        y = model(x)
        loss = criterion(y, x)
        loss.backward()
        optimizer.step()
        if batch_idx % 1 == 0:
            print('Epoch {}, iter {}: Loss = {}'.format(
            epoch, batch_idx, loss.item()))

Epoch 0, iter 0: Loss = 3.79117488861084
Epoch 0, iter 1: Loss = 3.7770721912384033
Epoch 0, iter 2: Loss = 3.7618191242218018
Epoch 0, iter 3: Loss = 3.7447564601898193
Epoch 0, iter 4: Loss = 3.724457025527954
Epoch 0, iter 5: Loss = 3.700133800506592
Epoch 0, iter 6: Loss = 3.6701197624206543
Epoch 0, iter 7: Loss = 3.633277416229248
Epoch 0, iter 8: Loss = 3.5877156257629395
Epoch 0, iter 9: Loss = 3.5311129093170166
Epoch 0, iter 10: Loss = 3.4612715244293213
Epoch 0, iter 11: Loss = 3.37591290473938
Epoch 0, iter 12: Loss = 3.2723517417907715
Epoch 0, iter 13: Loss = 3.1484875679016113
Epoch 0, iter 14: Loss = 3.0036075115203857
Epoch 0, iter 15: Loss = 2.8390984535217285
Epoch 0, iter 16: Loss = 2.660007953643799
Epoch 0, iter 17: Loss = 2.472672939300537
Epoch 0, iter 18: Loss = 2.2858076095581055
Epoch 0, iter 19: Loss = 2.1072564125061035
Epoch 0, iter 20: Loss = 1.9425816535949707
Epoch 0, iter 21: Loss = 1.7953356504440308
Epoch 0, iter 22: Loss = 1.6659270524978638
Epoch 0

Epoch 0, iter 186: Loss = 0.7531290650367737
Epoch 0, iter 187: Loss = 0.753006100654602
Epoch 0, iter 188: Loss = 0.7528464198112488
Epoch 0, iter 189: Loss = 0.7526193857192993
Epoch 0, iter 190: Loss = 0.7525658011436462
Epoch 0, iter 191: Loss = 0.7523863911628723
Epoch 0, iter 192: Loss = 0.7521646022796631
Epoch 0, iter 193: Loss = 0.7520661354064941
Epoch 0, iter 194: Loss = 0.7518808245658875
Epoch 0, iter 195: Loss = 0.7518708109855652
Epoch 0, iter 196: Loss = 0.7516417503356934
Epoch 0, iter 197: Loss = 0.7515733242034912
Epoch 0, iter 198: Loss = 0.7514220476150513
Epoch 0, iter 199: Loss = 0.7512821555137634
Epoch 0, iter 200: Loss = 0.7512460350990295
Epoch 0, iter 201: Loss = 0.7511410713195801
Epoch 0, iter 202: Loss = 0.75092613697052


KeyboardInterrupt: 

In [ ]:
len(train_dl)